### 1.Import Library

In [11]:
import pandas as pd

### 2.Explore Data Analysis

> data origin

In [12]:
df_bh=pd.read_csv("./data/bhxh_bhyt_bhtn_data.csv")
df_chinhsach=pd.read_csv("./data/chinhsach_nguoicocong_data.csv")
df_gd_daotao_yte=pd.read_csv("./data/giaoduc_daotao_yte_data.csv")
df_gt_xd_tn_mt=pd.read_csv("./data/giaothong_xaydung_tainguyen_moitruong_data.csv")
df_laodong_tt=pd.read_csv("./data/laodong_tienthuong_data.csv")
df_tc_nh_dt_ct=pd.read_csv("./data/taichinh_nganhang_dautu_congthuong_data.csv")
df_linhvuckhac=pd.read_csv("./data/linhvuckhac_data.csv")

> merge data

In [13]:
# Nối các DataFrame lại thành một DataFrame duy nhất
df_tonghop = pd.concat([df_bh, df_chinhsach, df_gd_daotao_yte, df_gt_xd_tn_mt, df_laodong_tt, df_tc_nh_dt_ct, df_linhvuckhac], ignore_index=True)

df_tonghop = df_tonghop.drop(columns='Index')

# Lưu DataFrame đã nối với việc thêm cột 'index' làm tên cột
df_tonghop.to_csv("./data/tonghop_data.csv", index_label='Index')


In [14]:
df_tonghop.shape

(12818, 5)

> Read CSV

In [15]:
df_train = pd.read_csv('./data/tonghop_data.csv')
df_alqac = pd.read_csv('./data/ALQAC.csv')

In [16]:
df_alqac.shape

(530, 3)

> Rename

In [17]:
df_train = df_train.rename(columns={'Question': 'question', 'Situation': 'situation', 'Answer': 'answer'})

> Create Context

In [18]:
df_train=df_train[['question', 'situation', 'answer']]
df_train['context']=df_train['situation']+df_train['answer']

> format

In [19]:
df_train=df_train[['context', 'question', 'answer']]

> To CSV

In [20]:
df_merged = pd.concat([df_train, df_alqac], axis=0)

In [21]:
df_merged.shape

(13348, 3)

> To CSV

In [22]:
import json
import csv
def json_to_csv(json_file_path, csv_writer):
    with open(json_file_path, 'r', encoding='utf-8') as json_file:
        data = json.load(json_file)
        for item in data:
            for entry in item['data']:
                for paragraph in entry['paragraphs']:
                    context = paragraph['context']
                    for qa in paragraph['qas']:
                        question = qa['question']
                        
                        # Kiểm tra và trích xuất câu trả lời
                        is_impossible = qa.get('is_impossible', False)
                        if not is_impossible:
                            answer = qa['answers'][0]['text'] if qa['answers'] else "No answer available"
                        else:
                            answer = "No answer available" 
                        csv_writer.writerow([context, question, answer])
with open('./data/qa_combined_data.csv', 'w', encoding='utf-8', newline='') as csv_file:
    writer = csv.writer(csv_file)
    writer.writerow(['context', 'question', 'answer'])
    json_to_csv('./data/qa_train.json', writer)
    json_to_csv('./data/eval.json', writer)

print("Chuyển đổi hoàn tất. Kiểm tra số dòng trong file CSV để đảm bảo có đủ 2824 dòng.")


Chuyển đổi hoàn tất. Kiểm tra số dòng trong file CSV để đảm bảo có đủ 2824 dòng.


In [23]:
import pandas as pd
data_git=pd.read_csv("./data/qa_combined_data.csv")

In [24]:
data_git.shape

(3977, 3)

In [25]:
data_git.isnull().sum()

context     0
question    0
answer      0
dtype: int64

>Final train

In [26]:
final_data= pd.concat([df_merged,data_git], axis=0)

In [27]:
final_data.shape

(17325, 3)

In [28]:
# import pandas as pd

# # Đọc từng file .parquet
# file1 = "./data/data/dieu-00000-of-00005.parquet"
# file2 = "./data/data/dieu-00001-of-00005.parquet"
# file3 = "./data/data/dieu-00002-of-00005.parquet"
# file4 = "./data/data/dieu-00003-of-00005.parquet"
# file5 = "./data/data/dieu-00004-of-00005.parquet"

# # Load dữ liệu từ các file
# df1 = pd.read_parquet(file1)
# df2 = pd.read_parquet(file2)
# df3 = pd.read_parquet(file3)
# df4 = pd.read_parquet(file4)
# df5 = pd.read_parquet(file5)

# # Ghép cả 5 DataFrame lại với nhau
# merged_df = pd.concat([df1, df2, df3, df4, df5], ignore_index=True)

# # Xuất ra file CSV
# output_file = "./data/bo_sung.csv"
# merged_df.to_csv(output_file, index=False)

# print(f"Đã ghép file thành công và lưu vào: {output_file}")


In [29]:
bo_sung_df=pd.read_csv("./data/bo_sung.csv")

In [30]:
bo_sung_df.shape

(10000, 4)

In [31]:
bo_sung_df.rename(columns={"text":"context","content":"answer","citation": "question"}, inplace=True)

In [32]:
formatted_df = bo_sung_df[["context", "question", "answer"]]

In [33]:
formatted_df.shape

(10000, 3)

In [34]:
final_data= pd.concat([final_data,formatted_df],axis=0)
final_data.shape

(27325, 3)

In [36]:
final_data.to_csv("./data/final_train.csv")

In [37]:
final_data.shape

(27325, 3)